In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time
import klib

In [2]:
df = pd.read_csv('KDDTrain+.csv')
# replace normal with 0 and anomaly with 1
df['class'] = df['class'].replace(['normal'], 0)
df['class'] = df['class'].replace(['anomaly'], 1)

In [3]:
df.drop(['duration', 'num_outbound_cmds'], axis=1, inplace=True)

In [4]:
# klib.data_cleaning(df)

In [5]:
#  in type, replace perl, xterm, buffer_overflow, ps, loadmodule, rootkit, sqlattack as u2r
df['type'] = df['type'].replace(['perl','xterm','buffer_overflow','ps','loadmodule','rootkit','sqlattack'], 'u2r')
# in type, replace warezclient, snmpguess, Snmpgetattack, phf, xsnoop, spy, named, warezmaster, http_tunnel, sendmail, imap, guess_passwd, ftp_write, multihop, xlock as r2l
df['type'] = df['type'].replace(['warezclient','snmpguess','snmpgetattack','phf','xsnoop','spy','named','warezmaster','httptunnel','sendmail','imap','guess_passwd','ftp_write','multihop','xlock'], 'r2l')
# in type, replace portsweep, nmap, saint, mscan, ipsweep, satan as probe
df['type'] = df['type'].replace(['portsweep','nmap','saint','mscan','ipsweep','satan'], 'probe')
# in type, replace mailbomb, worm, land, smurf, teardrop, processtable, back, pod, apache2, udpstorm, neptune as dos
df['type'] = df['type'].replace(['mailbomb','worm','land','smurf','teardrop','processtable','back','pod','apache2','udpstorm','neptune'], 'dos')

In [6]:
df['type'].value_counts()

type
normal    67343
dos       45927
probe     11656
r2l         995
u2r          52
Name: count, dtype: int64

In [7]:
cols_cat = df.select_dtypes('object').columns
cols_numeric = df._get_numeric_data().columns

def Remove_dump_values(data, cols):
    for col in cols:
        data[col] = np.where(data[col] == '-', 'None', data[col])
    return data

cols = df.columns
data_bin = Remove_dump_values(df, cols)
data_bin_attackcat = data_bin[['type']]

from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder(categories='auto')
oe.fit(data_bin_attackcat)
data_bin_attackcat = oe.transform(data_bin_attackcat)

data_bin.drop(['type'], axis=1, inplace=True)
cols_cat = cols_cat.drop(['type'])
data_bin_hot = pd.get_dummies(data_bin,columns=cols_cat)
attack_cat_df = pd.DataFrame(data_bin_attackcat, columns=['type'])
data_bin_hot2 = pd.concat([data_bin_hot,attack_cat_df],axis=1, join='inner',ignore_index=False)

data_bin_hot2[cols_numeric] = data_bin_hot2[cols_numeric].astype('float') 
# normalize cols_numeric
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_bin_hot2[cols_numeric] = scaler.fit_transform(data_bin_hot2[cols_numeric])
df = data_bin_hot2.copy()

In [8]:
# drop any rows that have NaN values
df = df.dropna()

In [9]:
df['type'].value_counts()

type
1.0    67343
0.0    45927
2.0    11656
3.0      995
4.0       52
Name: count, dtype: int64

In [10]:
# use smote to balance the dataset
from imblearn.over_sampling import SMOTE

X = df.drop('type', axis=1)
y = df['type']

smote = SMOTE()
X_sm, y_sm = smote.fit_resample(X, y)

df = pd.concat([X_sm, y_sm], axis=1)


In [11]:
df['type'].value_counts()

type
1.0    67343
0.0    67343
3.0    67343
2.0    67343
4.0    67343
Name: count, dtype: int64

In [ ]:
#save model
df.to_csv('KDDTrain+_cleaned.csv', index=False)